In [ ]:
# CODE FOR CREATING AND TRAINING CHATBOT

# import libraries
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# load json file with tags, patterns & responses

intents = json.loads(open("intents.json").read())

In [ ]:
# create lists to store data
words = []
labels = []
docs = []
ignore_letters = ["?", "!", ".", ","]

# separate words from patterns and add to words list
for intent in intents['intents']:
	for pattern in intent['patterns']:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list) 
		
		# associate patterns with respective tags
		docs.append(((word_list), intent['tag']))

		# append tags to labels list
		if intent['tag'] not in labels:
			labels.append(intent['tag'])

In [ ]:
# store root words (lemma)
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word)
		for word in words if word not in ignore_letters]
words = sorted(set(words))

In [ ]:
# save words and labels lists to binary files

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(labels, open('labels.pkl', 'wb'))

In [ ]:
# binarise data for neural network processing:
# (0 = word not in pattern, 1 = word in pattern)

training = []
output_empty = [0]*len(labels)
for doc in docs:
	bag = []
	word_patterns = doc[0]
	word_patterns = [lemmatizer.lemmatize(
		word.lower()) for word in word_patterns]
	for word in words:
		bag.append(1) if word in word_patterns else bag.append(0)
  # shuffle training data
	output_row = list(output_empty)
	output_row[labels.index(doc[1])] = 1
	training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)

<ipython-input-6-94b4e9f946bf>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# split data
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [ ]:
# create Sequential Neural Network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ),
				activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
# compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
			optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
				epochs=200, batch_size=5, verbose=1)

print("Training complete")

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
4/4 [==============================] - 2s 20ms/step - loss: 1.3855 - accuracy: 0.3529
Epoch 2/200
4/4 [==============================] - 0s 7ms/step - loss: 1.3900 - accuracy: 0.1765
Epoch 3/200
4/4 [==============================] - 0s 8ms/step - loss: 1.2978 - accuracy: 0.2941
Epoch 4/200
4/4 [==============================] - 0s 6ms/step - loss: 1.2980 - accuracy: 0.2941
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 1.2490 - accuracy: 0.4118
Epoch 6/200
4/4 [==============================] - 0s 10ms/step - loss: 1.2207 - accuracy: 0.3529
Epoch 7/200
4/4 [==============================] - 0s 9ms/step - loss: 1.1057 - accuracy: 0.4118
Epoch 8/200
4/4 [==============================] - 0s 7ms/step - loss: 1.2385 - accuracy: 0.3529
Epoch 9/200
4/4 [==============================] - 0s 10ms/step - loss: 1.1497 - accuracy: 0.3529
Epoch 10/200
4/4 [==============================] - 0s 10ms/step - loss: 1.1962 - accuracy: 0.4118
Epoch 11/200
4/4 [======

In [ ]:
# saving the model
model.save("chatbotmodel.h5", hist)